In [47]:
!pip install pandas Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.9 MB/s eta 0:00:00


In [48]:
# imports
import pandas as pd
import ast
import Levenshtein

# **Assignment 4 - Clustering and Recommendations**

Group: #45

Isaac Lafond - 300191954

**Dataset description:**

This dataset by Rounak Banik, contains 2 csv files: 1 containing movie metadata and a second containing movie ratings. The metadata contains a large list of attributes describing the movies:
*   belongs_to_collection: JSON string data on the collection that the movie is part of.
*   budget: An interger values of the movie's bduget ($).
*   genres: JSON string data on the genre of the movie (id and name).
*   id: Unique identifier of the movie in the dataset.
*   original_language: original language of the movie.
*   original_tilte: original title of the movie.
*   popularity: number representing the movie's popularity.
*   production_companies: JSON data on the production company (name and id).
*   production_countries: JSON data on the country the movie was produced in.
*   release_date: YYYY-MM-DD date of the movie's release
*   revenue: number represeting the revenue generated by the movie.
*   runtime: float of the movie's runtime.
*   spoken_languages: JSON data on the languages spoken in the movie (abbreveation and name).
*   title: string of the movie title.
*   vote_average: average of the rating given to the movie.
*   vote_count: number of of ratings given to movie.

The ratings contains a few more attributes:
*   userId: identifier of the user leaving the rating.
*   movieId: identifier of the movie being rates.
*   rating: rating given by the user.
*   timestamp: integer of the timestamp.

In [4]:
df_movies = pd.read_csv("https://raw.githubusercontent.com/IsaacLafond/CSI-4142---Fundamentals-of-Data-Science/main/Assignment%204/datasets/movies_metadata.csv")
df_ratings = pd.read_csv("https://raw.githubusercontent.com/IsaacLafond/CSI-4142---Fundamentals-of-Data-Science/main/Assignment%204/datasets/ratings_small.csv")

## **Data Preparation**

In [5]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45463 non-null  int64  
 2   genres                 45463 non-null  object 
 3   id                     45463 non-null  int64  
 4   original_language      45452 non-null  object 
 5   original_title         45463 non-null  object 
 6   popularity             45460 non-null  float64
 7   production_companies   45460 non-null  object 
 8   production_countries   45460 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  title                  45460 non-null  object 
 14  vote_average           45460 non-null  float64
 15  vo

In [6]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [45]:
# data cleaning
df_movies_clean = df_movies.copy()

# utils
# exract values from json data
def extract_values(extract_name, data):
  # convert string data to python list of dict
  if not isinstance(data, str):
    return data
  else:
    d = ast.literal_eval(data)
    if isinstance(d, list):
      j = []
      for item in d:
        j.append(item[extract_name])
      return j
    else:
      try:
        return d[extract_name]
      except:
        print(f"Error: {d}")
        return data

# 0: clean "belongs_to_collection" col (extract names)
df_movies_clean['belongs_to_collection'] = df_movies_clean['belongs_to_collection'].apply(lambda x: extract_values('name', x)).astype("string")

# 1: clean "budget" col (already int64 no non-null)

# 2: clean "genres" col (extract names from dict to list)
df_movies_clean['genres'] = df_movies_clean['genres'].apply(lambda x: extract_values('name', x))

# 3: clean "id" col (already int64 and no non-null)

# 4: clean "original_language" col
df_movies_clean['original_language'] = df_movies_clean['original_language'].astype("string")

# 5: clean "original_title" col
df_movies_clean['original_title'] = df_movies_clean['original_title'].astype("string")

# 6: clean "popularity" col (already float64)

# 7: clean "production_companies" col
df_movies_clean['production_companies'] = df_movies_clean['production_companies'].apply(lambda x: extract_values('name', x))

# 8: clean "production_countries" col
df_movies_clean['production_countries'] = df_movies_clean['production_countries'].apply(lambda x: extract_values('name', x))

# 9: clean "realease_date" col
# YYYY-MM-DD regex
regex = r'^\d{4}\-(0[1-9]|1[012])\-(0[1-9]|[12][0-9]|3[01])$'
# check that all values follow follow the regex
print(f"Values that don't match date format: {df_movies_clean[~df_movies_clean['release_date'].str.match(regex, na=False)]['release_date'].unique()}\n")
# convert values to datetime and leave nan as is
df_movies_clean['release_date'] = pd.to_datetime(df_movies_clean['release_date'], errors='coerce')

# 10: clean "revenue" col (already float64 no non-null)

# 11: clean "runtime" col (already float64)

# 12: clean "spoken_languages" col (extract name)
df_movies_clean['spoken_languages'] = df_movies_clean['spoken_languages'].apply(lambda x: extract_values('name', x))

# 13: clean "title" col
df_movies_clean['title'] = df_movies_clean['title'].astype("string")

# 14: clean "vote_average" col (already float64 no non-null)

# 15: clean "vote_count" col (already float64 no non-null)

print(df_movies_clean.info())
df_movies_clean.head()

Values that don't match date format: [nan]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4491 non-null   string        
 1   budget                 45463 non-null  int64         
 2   genres                 45463 non-null  object        
 3   id                     45463 non-null  int64         
 4   original_language      45452 non-null  string        
 5   original_title         45463 non-null  string        
 6   popularity             45460 non-null  float64       
 7   production_companies   45460 non-null  object        
 8   production_countries   45460 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45460 non-null  float64       
 11  runtime                45203 non-null  float64       
 12  spoken_languages

,belongs_to_collection,budget,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,Toy Story,21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Toy Story,7.7,5415.0
1,<NA>,65000000,"[Adventure, Fantasy, Family]",8844,en,Jumanji,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Jumanji,6.9,2413.0
2,Grumpy Old Men Collection,0,"[Romance, Comedy]",15602,en,Grumpier Old Men,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Grumpier Old Men,6.5,92.0
3,<NA>,16000000,"[Comedy, Drama, Romance]",31357,en,Waiting to Exhale,3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,[English],Waiting to Exhale,6.1,34.0
4,Father of the Bride Collection,0,[Comedy],11862,en,Father of the Bride Part II,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,106.0,[English],Father of the Bride Part II,5.7,173.0


## **Study 1 - Similarity measures**

In [147]:
# jaccard similarity
def jaccard_similarity(list1, list2):
  s1 = set(list1)
  s2 = set(list2)
  # handle empty sets
  if not s1 and not s2:
    return 1
  if not s1 or not s2:
    return 0
  return len(s1.intersection(s2)) / len(s1.union(s2))

# normalized edit distance (ie. levenshtein)
def edit_distance(s1, s2):
  if not s1 and not s2:
        return 1.0
  if not s1 or not s2:
      return 0.0

  # Calculate Levenshtein distance
  distance = Levenshtein.distance(s1.lower(), s2.lower())

  # Normalize by max length
  max_len = max(len(s1), len(s2))
  if max_len == 0:
      return 1.0

  # Convert to similarity (1 - normalized distance)
  similarity = 1 - (distance / max_len)
  return similarity

# normalized euclidean distance
def euclidean_distance(val1, val2, min_val, max_val):
    if pd.isna(val1) or pd.isna(val2):
        return 0.0

    # Normalize values to [0,1] range to ensure fair comparison
    range_val = max_val - min_val
    if range_val == 0:
        return 1.0 if val1 == val2 else 0.0

    norm_val1 = (val1 - min_val) / range_val
    norm_val2 = (val2 - min_val) / range_val

    # Calculate Euclidean distance and convert to similarity
    distance = abs(norm_val1 - norm_val2)
    similarity = 1 - distance

    return similarity

# manhattan distance
def manhattan_distance(val1, val2, min_val, max_val):
    # For a single dimension, Manhattan and Euclidean are the same
    return euclidean_distance(val1, val2, min_val, max_val)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
import ast
import re
import Levenshtein
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')

# Download NLTK resources (uncomment if needed)
# nltk.download('punkt')
# nltk.download('stopwords')

# Read the data
# In a real scenario, you would use:
# df = pd.read_csv('movies_data.csv')
# But for this exercise, I'll use the provided data as a string
data_string = """adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415
False,,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",,8844,tt0113497,en,Jumanji,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",,15602,tt0113228,en,Grumpier Old Men,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92
False,,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34
False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSAn204.jpg', 'backdrop_path': '/7qwE57OVZmMJChBpLEbJEmzUydk.jpg'}",0,"[{'id': 35, 'name': 'Comedy'}]",,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.",8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173
False,,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}]",,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a top-notch crew on various insane heists throughout Los Angeles while a mentally unstable detective, Vincent Hanna pursues him without rest. Each man recognizes and respects the ability and the dedication of the other even though they are aware their cat-and-mouse game may end in violence.",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {'name': 'Forward Pass', 'id': 675}, {'name': 'Warner Bros.', 'id': 6194}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,187436818,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'es', 'name': 'Español'}]",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886
False,,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]",,11860,tt0114319,en,Sabrina,"An ugly duckling having undergone a remarkable change, still harbors feelings for her crush: a carefree playboy, but not before his business-focused brother has something to say about it.",6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Scott Rudin Productions', 'id': 258}, {'name': 'Mirage Enterprises', 'id': 932}, {'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Constellation Entertainment', 'id': 14941}, {'name': 'Worldwide', 'id': 55873}, {'name': 'Mont Blanc Entertainment GmbH', 'id': 58079}]","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'iso_639_1': 'en', 'name': 'English'}]",Released,You are cordially invited to the most surprising merger of the year.,Sabrina,False,6.2,141
False,,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}]",,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses a murder by the deadly Injun Joe. Tom becomes friends with Huckleberry Finn, a boy with no future and no family. Tom has to choose between honoring a friendship or honoring an oath because the town alcoholic is accused of the murder. Tom and Huck go through several adventures trying to retrieve evidence.",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45
False,,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 53, 'name': 'Thriller'}]",,9091,tt0114576,en,Sudden Death,"International action superstar Jean Claude Van Damme teams with Powers Boothe in a Tension-packed, suspense thriller, set against the back-drop of a Stanley Cup game.Van Damme portrays a father whose daughter is suddenly taken during a championship hockey game. With the captors demanding a billion dollars by game's end, Van Damme frantically sets a plan in motion to rescue his daughter and abort an impending explosion before the final buzzer...",5.23158,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'name': 'Imperial Entertainment', 'id': 21437}, {'name': 'Signature Entertainment', 'id': 23770}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,64350171,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174"""

import io
df = pd.read_csv(io.StringIO(data_string))

# Clean and preprocess data
# Fix budget and revenue columns
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')

# Parse genres
def extract_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return [g['name'] for g in genres]
    except:
        return []

df['genre_list'] = df['genres'].apply(extract_genres)

# Clean overview text
def clean_text(text):
    if isinstance(text, str):
        # Remove special characters
        text = re.sub(r'[^\w\s]', ' ', text)
        # Convert to lowercase
        text = text.lower()
        # Tokenize
        tokens = word_tokenize(text)
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [t for t in tokens if t not in stop_words]
        return tokens
    return []

df['overview_tokens'] = df['overview'].apply(clean_text)

# Add a sample movie (Titanic) for the revenue similarity example
sample_titanic = {
    'title': 'Titanic',
    'budget': 200000000,
    'revenue': 1845000000,
    'runtime': 194.0,
    'overview': "84 years later, a 100 year-old woman named Rose DeWitt Bukater tells the story to her granddaughter Lizzy Calvert, Brock Lovett, Lewis Bodine, Bobby Buell and Anatoly Mikailavich on the Kelpie about her life set in April 10th 1912, on a ship called Titanic when young Rose boards the departing ship with the upper-class passengers and her mother, Ruth DeWitt Bukater, and her fiancé, Caledon Hockley. Meanwhile, a drifter and artist named Jack Dawson and his best friend Fabrizio De Rossi win third-class tickets to the ship in a game. And she explains the whole story from departure until the death of Titanic on its first and last voyage April 15th, 1912 at 2:20 in the morning.",
    'genre_list': ['Drama', 'Romance', 'Thriller']
}

# Add Apollo 13 for runtime similarity
sample_apollo = {
    'title': 'Apollo 13',
    'budget': 52000000,
    'revenue': 355000000,
    'runtime': 140.0,
    'overview': "The true story of technical troubles that scuttle the Apollo 13 lunar mission in 1971, risking the lives of astronaut Jim Lovell and his crew, with the failed journey turning into a thrilling saga of heroism. Drifting more than 200,000 miles from Earth, the astronauts work furiously with the ground crew to avert tragedy.",
    'genre_list': ['Drama', 'Adventure', 'History']
}

# Add Fight Club for title similarity
sample_fightclub = {
    'title': 'Fight Club',
    'budget': 63000000,
    'revenue': 100900000,
    'runtime': 139.0,
    'overview': "A ticking-time-bomb insomniac and a slippery soap salesman channel primal male aggression into a shocking new form of therapy. Their concept catches on, with underground 'fight clubs' forming in every town, until an eccentric gets in the way and ignites an out-of-control spiral toward oblivion.",
    'genre_list': ['Drama', 'Thriller', 'Action']
}

# Add Matrix for budget similarity
sample_matrix = {
    'title': 'The Matrix',
    'budget': 63000000,
    'revenue': 463500000,
    'runtime': 136.0,
    'overview': "Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.",
    'genre_list': ['Action', 'Science Fiction']
}

# Now let's implement our similarity functions
def jaccard_similarity(list1, list2):
    """Implement Jaccard similarity for categorical data like genres"""
    set1 = set(list1)
    set2 = set(list2)

    # Handle empty sets
    if not set1 and not set2:
        return 1.0
    if not set1 or not set2:
        return 0.0

    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))

    return intersection / union

def cosine_text_similarity(tokens1, tokens2):
    """
    Compute cosine similarity between two sets of text tokens
    using a simple bag-of-words approach
    """
    # Create vocabulary from both token lists
    all_tokens = set(tokens1 + tokens2)

    # Create vector representations
    vec1 = np.array([1 if t in tokens1 else 0 for t in all_tokens])
    vec2 = np.array([1 if t in tokens2 else 0 for t in all_tokens])

    # Compute cosine similarity
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)

    if norm1 == 0 or norm2 == 0:
        return 0.0

    return dot_product / (norm1 * norm2)

def normalized_levenshtein_distance(s1, s2):
    """
    Calculate normalized Levenshtein distance (edit distance)
    for text similarity. Returns a similarity score (1 - normalized distance).
    """
    if not s1 and not s2:
        return 1.0
    if not s1 or not s2:
        return 0.0

    # Calculate Levenshtein distance
    distance = Levenshtein.distance(s1.lower(), s2.lower())

    # Normalize by max length
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0

    # Convert to similarity (1 - normalized distance)
    similarity = 1 - (distance / max_len)
    return similarity

def normalized_euclidean_distance(val1, val2, min_val, max_val):
    """Calculate normalized Euclidean distance for numerical values"""
    if pd.isna(val1) or pd.isna(val2):
        return 0.0

    # Normalize values to [0,1] range to ensure fair comparison
    range_val = max_val - min_val
    if range_val == 0:
        return 1.0 if val1 == val2 else 0.0

    norm_val1 = (val1 - min_val) / range_val
    norm_val2 = (val2 - min_val) / range_val

    # Calculate Euclidean distance and convert to similarity
    distance = abs(norm_val1 - norm_val2)
    similarity = 1 - distance

    return similarity

def normalized_manhattan_distance(val1, val2, min_val, max_val):
    """Calculate normalized Manhattan distance for numerical values"""
    # For a single dimension, Manhattan and Euclidean are the same
    return normalized_euclidean_distance(val1, val2, min_val, max_val)

# Function to find similar movies based on genres (Jaccard similarity)
def find_similar_genres(query_movie, df, top_n=10):
    """Find movies with similar genres using Jaccard similarity"""
    query_genres = query_movie['genre_list']

    # Calculate similarity for each movie
    similarities = []
    for idx, row in df.iterrows():
        sim = jaccard_similarity(query_genres, row['genre_list'])
        similarities.append((idx, sim))

    # Sort by similarity (descending)
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Get top N results
    top_indices = [idx for idx, sim in similarities[:top_n]]

    # Create result DataFrame
    result_df = df.iloc[top_indices].copy()
    result_df['similarity_score'] = [sim for _, sim in similarities[:top_n]]

    # Select columns to display
    display_cols = ['title', 'genre_list', 'similarity_score', 'popularity']

    return result_df[display_cols].sort_values('similarity_score', ascending=False)

# Function to find similar movies based on revenue (Euclidean distance)
def find_similar_revenue(query_movie, df, top_n=10):
    """Find movies with similar revenue using normalized Euclidean distance"""
    query_revenue = query_movie['revenue']

    # Filter out movies with unknown revenue
    valid_df = df[df['revenue'] > 0].copy()

    if valid_df.empty:
        return pd.DataFrame()

    # Get min and max values for normalization
    min_revenue = min(valid_df['revenue'].min(), query_revenue)
    max_revenue = max(valid_df['revenue'].max(), query_revenue)

    # Calculate similarity for each movie
    similarities = []
    for idx, row in valid_df.iterrows():
        sim = normalized_euclidean_distance(
            query_revenue, row['revenue'], min_revenue, max_revenue)
        similarities.append((idx, sim))

    # Sort by similarity (descending)
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Get top N results
    top_indices = [idx for idx, sim in similarities[:top_n]]

    # Create result DataFrame
    result_df = valid_df.iloc[top_indices].copy()
    result_df['similarity_score'] = [sim for _, sim in similarities[:top_n]]

    # Select columns to display
    display_cols = ['title', 'revenue', 'similarity_score', 'popularity']

    return result_df[display_cols].sort_values('similarity_score', ascending=False)

# Function to find similar movies based on runtime (Manhattan distance)
def find_similar_runtime(query_movie, df, top_n=10):
    """Find movies with similar runtime using normalized Manhattan distance"""
    query_runtime = query_movie['runtime']

    # Filter out movies with unknown runtime
    valid_df = df[df['runtime'] > 0].copy()

    if valid_df.empty:
        return pd.DataFrame()

    # Get min and max values for normalization
    min_runtime = min(valid_df['runtime'].min(), query_runtime)
    max_runtime = max(valid_df['runtime'].max(), query_runtime)

    # Calculate similarity for each movie
    similarities = []
    for idx, row in valid_df.iterrows():
        sim = normalized_manhattan_distance(
            query_runtime, row['runtime'], min_runtime, max_runtime)
        similarities.append((idx, sim))

    # Sort by similarity (descending)
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Get top N results
    top_indices = [idx for idx, sim in similarities[:top_n]]

    # Create result DataFrame
    result_df = valid_df.iloc[top_indices].copy()
    result_df['similarity_score'] = [sim for _, sim in similarities[:top_n]]

    # Select columns to display
    display_cols = ['title', 'runtime', 'similarity_score', 'popularity']

    return result_df[display_cols].sort_values('similarity_score', ascending=False)

# Function to find similar movies based on title (Levenshtein/Edit distance)
def find_similar_titles(query_movie, df, top_n=10):
    """Find movies with similar titles using Levenshtein (edit) distance"""
    query_title = query_movie['title']

    # Calculate similarity for each movie
    similarities = []
    for idx, row in df.iterrows():
        sim = normalized_levenshtein_distance(query_title, row['title'])
        similarities.append((idx, sim))

    # Sort by similarity (descending)
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Get top N results
    top_indices = [idx for idx, sim in similarities[:top_n]]

    # Create result DataFrame
    result_df = df.iloc[top_indices].copy()
    result_df['similarity_score'] = [sim for _, sim in similarities[:top_n]]

    # Select columns to display
    display_cols = ['title', 'similarity_score', 'popularity']

    return result_df[display_cols].sort_values('similarity_score', ascending=False)

# Function to find similar movies based on budget (Cosine similarity)
def find_similar_budget(query_movie, df, top_n=10):
    """Find movies with similar budget using cosine similarity"""
    query_budget = query_movie['budget']

    # Filter out movies with unknown budget
    valid_df = df[df['budget'] > 0].copy()

    if valid_df.empty:
        return pd.DataFrame()

    # Create array of budgets
    budgets = valid_df['budget'].values.reshape(-1, 1)
    query_budget_array = np.array([[query_budget]])

    # Calculate cosine similarity
    similarities = cosine_similarity(query_budget_array, budgets)[0]

    # Create a list of (index, similarity) tuples
    sim_tuples = list(zip(valid_df.index, similarities))

    # Sort by similarity (descending)
    sim_tuples.sort(key=lambda x: x[1], reverse=True)

    # Get top N results
    top_indices = [idx for idx, sim in sim_tuples[:top_n]]

    # Create result DataFrame
    result_df = valid_df.iloc[top_indices].copy()
    result_df['similarity_score'] = [sim for _, sim in sim_tuples[:top_n]]

    # Select columns to display
    display_cols = ['title', 'budget', 'similarity_score', 'popularity']

    return result_df[display_cols].sort_values('similarity_score', ascending=False)

# Now let's simulate the requests

# Find movies with genres similar to Toy Story
# We'll use the first movie in our dataset which is Toy Story
toy_story = df.iloc[0].to_dict()

print("\n===== Movies with similar genres to Toy Story (Jaccard Similarity) =====")
similar_genres = find_similar_genres(toy_story, df, top_n=10)
print(similar_genres)

# Find movies with revenue similar to Titanic
print("\n===== Movies with similar revenue to Titanic (Normalized Euclidean Distance) =====")
similar_revenue = find_similar_revenue(sample_titanic, df, top_n=10)
print(similar_revenue)

# Find movies with runtime similar to Apollo 13
print("\n===== Movies with similar runtime to Apollo 13 (Normalized Manhattan Distance) =====")
similar_runtime = find_similar_runtime(sample_apollo, df, top_n=10)
print(similar_runtime)

# Find movies with titles similar to Fight Club (Levenshtein/Edit Distance)
print("\n===== Movies with similar titles to Fight Club (Edit Distance) =====")
similar_titles = find_similar_titles(sample_fightclub, df, top_n=10)
print(similar_titles)

# Find movies with budget similar to The Matrix (Cosine Similarity)
print("\n===== Movies with similar budget to The Matrix (Cosine Similarity) =====")
similar_budget = find_similar_budget(sample_matrix, df, top_n=10)
print(similar_budget)

## **Study 2 - Clustering Algorithms**

## **Study 3 - Content-Based Recommendation System**

## **Study 4 - Collaborative Filtering Recommendation System**

## **Conclusion**

## **References**